In [15]:
from google.colab import drive
drive.mount('/content/drive')
drive_path = 'drive/MyDrive/imitate-retrieve-paraphrase'

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [16]:
%pip install datasets
%pip install transformers
%pip install accelerate -U

In [17]:
from datasets import Dataset
import pandas as pd
from transformers import TextDataset, DataCollatorForLanguageModeling
from transformers import GPT2Tokenizer, GPT2LMHeadModel
from transformers import TrainingArguments, Trainer

In [18]:
ds = Dataset.from_file(drive_path + "/medline_data.arrow")

In [19]:
expository_text_corpus = []

for row in ds:
    expository_text_corpus.append(row['output_aug'])

pd.DataFrame(expository_text_corpus).to_csv(drive_path + "/imitator/expository_text_corpus.csv", index = False)
text_data = open(drive_path + '/imitator/expository_text_corpus.txt', 'w')

for row in ds:
  text = row["output_aug"]
  text_data.write(text)
  text_data.write(' ')

text_data.close()

In [20]:
tokenizer = GPT2Tokenizer.from_pretrained('gpt2-large')
model = GPT2LMHeadModel.from_pretrained('gpt2-large').to('cuda')
train_dataset = TextDataset(tokenizer = tokenizer, file_path = drive_path +'/imitator/expository_text_corpus.txt', block_size = 128)
data_collator = DataCollatorForLanguageModeling(tokenizer = tokenizer, mlm = False)

training_args = TrainingArguments(
        output_dir = drive_path + '/imitator',
        overwrite_output_dir = False,
        per_device_train_batch_size = 8,
        num_train_epochs = 5.0
)

trainer = Trainer(
        model = model,
        args = training_args,
        data_collator = data_collator,
        train_dataset = train_dataset
)

In [21]:
trainer.train()
trainer.save_model(drive_path + '/imitator/model')

Step,Training Loss
